In [22]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("data.json").read()
intents = json.loads(data_file)

# 2. Data pre-processing

In [26]:
# preprocessing the json data
# tokenization
#nltk.download('punkt')
#nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### Tokenisasi



In [27]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('texts.pkl','wb')) 
pickle.dump(classes,open('labels.pkl','wb'))

156 documents
38 classes ['breast_cancer1', 'breast_cancer2', 'breast_cancer3', 'endometriosis1', 'fertility1', 'fertility2', 'fertility3', 'fibroid1', 'greeting1', 'greeting2', 'greeting3', 'greeting_malam', 'greeting_sore', 'infeksi1', 'kanker_serviks1', 'kb1', 'menopause1', 'menopause2', 'menopause3', 'menstruasi1', 'noanswer', 'osteoporosis1', 'ovulasi1', 'payudara_bengkak1', 'pcos1', 'pengalaman1', 'pengalaman2', 'pengalaman3', 'penutup1', 'penutup2', 'perdarahan_postmenopause1', 'polip1', 'pregnancy1', 'pregnancy2', 'pregnancy3', 'sts1', 'tingkat_hormon1', 'vulvitis1']
62 unique lemmatized words ['awal', 'bengkak', 'deteksi', 'dini', 'endometriosis', 'faktor', 'fertilisasi', 'gejala', 'hai', 'halo', 'hello', 'hormon', 'in', 'infeksi', 'kanker', 'kasih', 'kehamilan', 'kencing', 'kesuburan', 'kontrasepsi', 'malam', 'malem', 'masalah', 'menopause', 'menstruasi', 'metode', 'mioma', 'osteoporosis', 'ovarium', 'ovulasi', 'payudara', 'pengobatan', 'penyakit', 'perawatan', 'perdarahan', 

# 3. Creating Training Data



In [28]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle features and converting it into numpy arrays
random.shuffle(training)

# Split training into two separate lists
train_x = [row[0] for row in training]
train_y = [row[1] for row in training]

# Convert the lists to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training data created")

Training data created


# 5. Creating Modeling 



In [29]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('models.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 9.9501e-04 - loss: 3.6968   
Epoch 2/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0317 - loss: 3.5808     
Epoch 3/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1171 - loss: 3.4991
Epoch 4/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1392 - loss: 3.3499     
Epoch 5/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2305 - loss: 3.2144 
Epoch 6/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1928 - loss: 3.1073 
Epoch 7/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3305 - loss: 2.7270 
Epoch 8/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3851 - loss: 2.5686 
Epoch 9/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5016 - loss: 2.3418 
Epoch 10/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5394 - loss: 1.9751 
Epoch 11/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5805 - loss: 1.8028 
Epoch 12/200
32/32 ━━━━━━━━━━━━━━━━━━━━



**************************************************

Model Created Successfully!
